In [ ]:
import math
import random
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy

random.seed(42)
np.random.seed(42)

INSTANCE_PATH = "instance/VRPLIB/tests/data/A-n32-k5.vrp"
INITIAL_TEMPERATURE = 1000
COOLING_RATE = 0.995
MAX_ITERATIONS = 10000
MIN_TEMPERATURE = 0.1

In [8]:
def parse_vrplib(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    data = {
        'name': '',
        'comment': '',
        'type': '',
        'dimension': 0,
        'capacity': 0,
        'edge_weight_type': '',
        'nodes': [],
        'demands': [],
        'depot': 1
    }
    
    section = None
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        if line.startswith('NAME'):
            data['name'] = line.split(':')[1].strip()
        elif line.startswith('COMMENT'):
            data['comment'] = line.split(':')[1].strip()
        elif line.startswith('TYPE'):
            data['type'] = line.split(':')[1].strip()
        elif line.startswith('DIMENSION'):
            data['dimension'] = int(line.split(':')[1].strip())
        elif line.startswith('CAPACITY'):
            data['capacity'] = int(line.split(':')[1].strip())
        elif line.startswith('EDGE_WEIGHT_TYPE'):
            data['edge_weight_type'] = line.split(':')[1].strip()
        elif line.startswith('NODE_COORD_SECTION'):
            section = 'nodes'
        elif line.startswith('DEMAND_SECTION'):
            section = 'demands'
        elif line.startswith('DEPOT_SECTION'):
            section = 'depot'
        elif line.startswith('EOF'):
            break
        elif section == 'nodes':
            parts = line.split()
            if len(parts) == 3:
                node_id, x, y = int(parts[0]), float(parts[1]), float(parts[2])
                data['nodes'].append((node_id, x, y))
        elif section == 'demands':
            parts = line.split()
            if len(parts) == 2:
                node_id, demand = int(parts[0]), int(parts[1])
                data['demands'].append((node_id, demand))
        elif section == 'depot':
            if line != '-1':
                data['depot'] = int(line)
    
    return data

In [9]:
class Client:
    def __init__(self, id, x, y, demand):
        self.id = id
        self.x = x
        self.y = y
        self.demand = demand

class Vehicle:
    def __init__(self, capacity):
        self.capacity = capacity
        self.route = []
        self.load = 0
    
    def add_client(self, client):
        if self.load + client.demand <= self.capacity:
            self.route.append(client)
            self.load += client.demand
            return True
        return False
    
    def remove_client(self, client):
        if client in self.route:
            self.route.remove(client)
            self.load -= client.demand

class Solution:
    def __init__(self, vehicles):
        self.vehicles = vehicles
        self.cost = 0
    
    def calculate_cost(self, depot):
        total_distance = 0
        for vehicle in self.vehicles:
            if len(vehicle.route) == 0:
                continue
            route_distance = euclidean_distance(depot, vehicle.route[0])
            for i in range(len(vehicle.route) - 1):
                route_distance += euclidean_distance(vehicle.route[i], vehicle.route[i + 1])
            route_distance += euclidean_distance(vehicle.route[-1], depot)
            total_distance += route_distance
        self.cost = total_distance
        return total_distance

def euclidean_distance(node1, node2):
    return math.sqrt((node1.x - node2.x)**2 + (node1.y - node2.y)**2)

In [10]:
def generate_initial_solution(clients, depot, num_vehicles, vehicle_capacity):
    vehicles = [Vehicle(vehicle_capacity) for _ in range(num_vehicles)]
    unassigned_clients = clients[:]
    random.shuffle(unassigned_clients)
    
    current_vehicle_idx = 0
    for client in unassigned_clients:
        assigned = False
        attempts = 0
        while not assigned and attempts < num_vehicles:
            if vehicles[current_vehicle_idx].add_client(client):
                assigned = True
            else:
                current_vehicle_idx = (current_vehicle_idx + 1) % num_vehicles
                attempts += 1
        
        if not assigned:
            vehicles.append(Vehicle(vehicle_capacity))
            vehicles[-1].add_client(client)
    
    solution = Solution(vehicles)
    solution.calculate_cost(depot)
    return solution

In [11]:
def generate_neighbor(solution, depot):
    new_solution = deepcopy(solution)
    
    move_type = random.choice(['swap', 'relocate', '2opt'])
    
    non_empty_vehicles = [v for v in new_solution.vehicles if len(v.route) > 0]
    
    if len(non_empty_vehicles) < 1:
        return new_solution
    
    if move_type == 'swap' and len(non_empty_vehicles) >= 2:
        v1, v2 = random.sample(non_empty_vehicles, 2)
        if len(v1.route) > 0 and len(v2.route) > 0:
            i1 = random.randint(0, len(v1.route) - 1)
            i2 = random.randint(0, len(v2.route) - 1)
            
            c1, c2 = v1.route[i1], v2.route[i2]
            
            if v1.load - c1.demand + c2.demand <= v1.capacity and \
               v2.load - c2.demand + c1.demand <= v2.capacity:
                v1.route[i1] = c2
                v2.route[i2] = c1
                v1.load = v1.load - c1.demand + c2.demand
                v2.load = v2.load - c2.demand + c1.demand
    
    elif move_type == 'relocate':
        v1 = random.choice(non_empty_vehicles)
        v2 = random.choice(new_solution.vehicles)
        
        if len(v1.route) > 0:
            i1 = random.randint(0, len(v1.route) - 1)
            client = v1.route[i1]
            
            if v2.load + client.demand <= v2.capacity:
                v1.route.pop(i1)
                v1.load -= client.demand
                
                if len(v2.route) > 0:
                    i2 = random.randint(0, len(v2.route))
                    v2.route.insert(i2, client)
                else:
                    v2.route.append(client)
                v2.load += client.demand
    
    elif move_type == '2opt':
        vehicle = random.choice(non_empty_vehicles)
        route = vehicle.route
        
        if len(route) > 3:
            i = random.randint(0, len(route) - 2)
            j = random.randint(i + 1, len(route) - 1)
            vehicle.route[i:j+1] = reversed(vehicle.route[i:j+1])
    
    new_solution.calculate_cost(depot)
    return new_solution

def acceptance_probability(current_cost, new_cost, temperature):
    if new_cost < current_cost:
        return 1.0
    return math.exp((current_cost - new_cost) / temperature)

def simulated_annealing(initial_solution, depot, initial_temp, cooling_rate, max_iter, min_temp):
    current_solution = initial_solution
    best_solution = deepcopy(current_solution)
    
    temperature = initial_temp
    iteration = 0
    
    while iteration < max_iter and temperature > min_temp:
        neighbor = generate_neighbor(current_solution, depot)
        
        current_cost = current_solution.cost
        neighbor_cost = neighbor.cost
        
        if acceptance_probability(current_cost, neighbor_cost, temperature) > random.random():
            current_solution = neighbor
            
            if current_solution.cost < best_solution.cost:
                best_solution = deepcopy(current_solution)
        
        temperature *= cooling_rate
        iteration += 1
    
    return best_solution

In [12]:
data = parse_vrplib(INSTANCE_PATH)

clients = []
for node in data['nodes']:
    node_id, x, y = node
    if node_id != data['depot']:
        demand = next((d[1] for d in data['demands'] if d[0] == node_id), 0)
        clients.append(Client(node_id, x, y, demand))

depot_node = next((node for node in data['nodes'] if node[0] == data['depot']), None)
depot = Client(depot_node[0], depot_node[1], depot_node[2], 0)

num_vehicles = 5
vehicle_capacity = data['capacity']

initial_solution = generate_initial_solution(clients, depot, num_vehicles, vehicle_capacity)
print(f"Solution initiale - Distance totale: {initial_solution.cost:.2f}")

best_solution = simulated_annealing(
    initial_solution, 
    depot, 
    INITIAL_TEMPERATURE, 
    COOLING_RATE, 
    MAX_ITERATIONS, 
    MIN_TEMPERATURE
)

print(f"Meilleure solution - Distance totale: {best_solution.cost:.2f}")
print(f"Nombre de véhicules utilisés: {sum(1 for v in best_solution.vehicles if len(v.route) > 0)}")

FileNotFoundError: [Errno 2] No such file or directory: 'A-n32-k5.vrp'

In [ ]:
plt.figure(figsize=(12, 8))

colors = plt.cm.tab10(np.linspace(0, 1, len(best_solution.vehicles)))

plt.scatter(depot.x, depot.y, c='red', s=200, marker='s', label='Dépôt', zorder=5)

for idx, vehicle in enumerate(best_solution.vehicles):
    if len(vehicle.route) == 0:
        continue
    
    route_x = [depot.x] + [client.x for client in vehicle.route] + [depot.x]
    route_y = [depot.y] + [client.y for client in vehicle.route] + [depot.y]
    
    plt.plot(route_x, route_y, c=colors[idx], linewidth=2, alpha=0.7, label=f'Véhicule {idx+1}')
    
    for client in vehicle.route:
        plt.scatter(client.x, client.y, c=[colors[idx]], s=100, zorder=3)
        plt.annotate(str(client.id), (client.x, client.y), fontsize=8, ha='center', va='center')

plt.title(f'Solution VRP - Distance totale: {best_solution.cost:.2f}', fontsize=14, fontweight='bold')
plt.xlabel('X', fontsize=12)
plt.ylabel('Y', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()